In [1]:
import numpy as np
import lightgbm as lgb
from sklearn.datasets import load_boston
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import metrics,ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error as MSLE
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import random
import csv
import pickle
from sklearn.preprocessing import StandardScaler
random.seed(123)
#pd.set_option("display.max_columns", None)
from sklearn.preprocessing import LabelEncoder
from sklearn import svm 
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

import os, time, glob, socket,random,csv,pickle
import pandas as pd

from timeit import Timer
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.decomposition import PCA 

pd.set_option("display.max_columns", None)
os.getcwd()
os.chdir(r'D:/fintech/')
path=os.getcwd()
import re

In [2]:
# A

y_train=pd.read_csv(path+'/grade/A/y_trainA.csv',index_col=0)
y_test=pd.read_csv(path+'/grade/A/y_testA.csv',index_col=0)
X_test=pd.read_csv(path+'/grade/A/X_testA.csv',index_col=0)
X_train=pd.read_csv(path+'/grade/A/X_trainA.csv',index_col=0)

In [3]:
print(y_train.shape)
print(y_test.shape)

(481870, 2)
(120505, 2)


imp=['Unnamed: 0.1','7','8','debt_settlement_flag_Y','20','0','12','9','11','hardship_flag_unknown']

X_test=X_test[imp]
X_train=X_train[imp]


In [4]:
X_trainall=pd.read_csv('X_train.csv')
X_testall=pd.read_csv('X_test.csv')

X_trainall=X_trainall.rename(columns={'Unnamed: 0':'Unnamed: 0.1'})
X_testall=X_testall.rename(columns={'Unnamed: 0':'Unnamed: 0.1'})

In [5]:
filter=X_trainall.grade=='A'
X_trainall_A=X_trainall[filter]


filter=X_testall.grade=='A'
X_testall_A=X_testall[filter]

In [6]:
#earliest_cr_line=借款人最早報告的信用額度被打開的月份
#issue_d=貸款資金的月份
#last_credit_pull_d=最近一個月 LC 為這筆貸款拉了貸
#last_pymnt_d=上個月收到付

p=['Unnamed: 0.1','emp_title','earliest_cr_line','issue_d','last_credit_pull_d','last_pymnt_d','addr_state']
X_train_add=X_trainall_A[p]


p=['Unnamed: 0.1','emp_title','earliest_cr_line','issue_d','last_credit_pull_d','last_pymnt_d','addr_state']
X_test_add=X_testall_A[p]


In [7]:
from datetime import datetime
import _strptime

In [8]:
all_d=X_train_add

all_d['last_credit_pull_d_trans'] = pd.to_datetime(all_d['last_credit_pull_d'], format= "%b-%y")
all_d['last_credit_pull_d_year'] = pd.DatetimeIndex(all_d['last_credit_pull_d_trans']).year
all_d['last_credit_pull_d_month'] =pd.DatetimeIndex(all_d['last_credit_pull_d_trans']).month
all_d.drop(columns=['last_credit_pull_d'])

all_d['last_pymnt_d_trans'] = pd.to_datetime(all_d['last_pymnt_d'], format= "%b-%y")
all_d['last_pymnt_d_year'] = pd.DatetimeIndex(all_d['last_pymnt_d_trans']).year
all_d['last_pymnt_d_month'] =pd.DatetimeIndex(all_d['last_pymnt_d_trans']).month
all_d.drop(columns=['last_pymnt_d'])

all_d['issue_d_trans'] = pd.to_datetime(all_d['issue_d'], format= "%b-%y")
all_d['issue_d_year'] = pd.DatetimeIndex(all_d['issue_d_trans']).year
all_d['issue_d_month'] =pd.DatetimeIndex(all_d['issue_d_trans']).month
all_d.drop(columns=['issue_d'])

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,Unnamed: 0.1,emp_title,earliest_cr_line,last_credit_pull_d,last_pymnt_d,addr_state,last_credit_pull_d_trans,last_credit_pull_d_year,last_credit_pull_d_month,last_pymnt_d_trans,last_pymnt_d_year,last_pymnt_d_month,issue_d_trans,issue_d_year,issue_d_month
0,99620,Visual/Audio Tech,Aug-02,Aug-20,Sep-16,FL,2020-08-01,2020.0,8.0,2016-09-01,2016.0,9.0,2015-10-01,2015,10
1,1676459,Principal,Dec-09,Aug-20,Aug-20,TX,2020-08-01,2020.0,8.0,2020-08-01,2020.0,8.0,2019-08-01,2019,8
3,2230019,Sr Analyst,Nov-05,May-19,Feb-19,PA,2019-05-01,2019.0,5.0,2019-02-01,2019.0,2.0,2018-09-01,2018,9
10,819438,Line lead,Apr-04,Oct-19,Sep-19,VA,2019-10-01,2019.0,10.0,2019-09-01,2019.0,9.0,2018-10-01,2018,10
12,1486093,Teacher,Mar-97,Aug-20,Sep-17,TX,2020-08-01,2020.0,8.0,2017-09-01,2017.0,9.0,2016-08-01,2016,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2132770,1482659,Manager,Jan-00,Aug-20,Aug-20,IL,2020-08-01,2020.0,8.0,2020-08-01,2020.0,8.0,2017-09-01,2017,9
2132775,739979,Commercial/Ag Lender,Aug-09,Aug-20,Aug-20,NE,2020-08-01,2020.0,8.0,2020-08-01,2020.0,8.0,2018-10-01,2018,10
2132778,1650754,Dispatcher,Sep-95,Aug-20,Feb-19,ID,2020-08-01,2020.0,8.0,2019-02-01,2019.0,2.0,2016-03-01,2016,3
2132784,2277769,Music Teacher,Aug-01,Aug-20,Mar-18,MA,2020-08-01,2020.0,8.0,2018-03-01,2018.0,3.0,2015-04-01,2015,4


In [9]:
all_d=X_test_add

all_d['last_credit_pull_d_trans'] = pd.to_datetime(all_d['last_credit_pull_d'], format= "%b-%y")
all_d['last_credit_pull_d_year'] = pd.DatetimeIndex(all_d['last_credit_pull_d_trans']).year
all_d['last_credit_pull_d_month'] =pd.DatetimeIndex(all_d['last_credit_pull_d_trans']).month
all_d.drop(columns=['last_credit_pull_d'])

all_d['last_pymnt_d_trans'] = pd.to_datetime(all_d['last_pymnt_d'], format= "%b-%y")
all_d['last_pymnt_d_year'] = pd.DatetimeIndex(all_d['last_pymnt_d_trans']).year
all_d['last_pymnt_d_month'] =pd.DatetimeIndex(all_d['last_pymnt_d_trans']).month
all_d.drop(columns=['last_pymnt_d'])

all_d['issue_d_trans'] = pd.to_datetime(all_d['issue_d'], format= "%b-%y")
all_d['issue_d_year'] = pd.DatetimeIndex(all_d['issue_d_trans']).year
all_d['issue_d_month'] =pd.DatetimeIndex(all_d['issue_d_trans']).month
all_d.drop(columns=['issue_d'])

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,Unnamed: 0.1,emp_title,earliest_cr_line,last_credit_pull_d,last_pymnt_d,addr_state,last_credit_pull_d_trans,last_credit_pull_d_year,last_credit_pull_d_month,last_pymnt_d_trans,last_pymnt_d_year,last_pymnt_d_month,issue_d_trans,issue_d_year,issue_d_month
1,951395,Personal trainer,Mar-00,Jul-17,Aug-16,FL,2017-07-01,2017.0,7.0,2016-08-01,2016.0,8.0,2016-05-01,2016,5
2,2322300,NaN,Dec-99,Apr-17,May-17,WI,2017-04-01,2017.0,4.0,2017-05-01,2017.0,5.0,2014-11-01,2014,11
11,1831731,Supervisor,Oct-06,Jul-20,Jul-17,NY,2020-07-01,2020.0,7.0,2017-07-01,2017.0,7.0,2015-04-01,2015,4
15,334694,Account Manager,Apr-05,Aug-20,Aug-20,CA,2020-08-01,2020.0,8.0,2020-08-01,2020.0,8.0,2020-04-01,2020,4
16,2617585,Patient Access,Sep-11,Aug-20,Aug-20,OH,2020-08-01,2020.0,8.0,2020-08-01,2020.0,8.0,2018-10-01,2018,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533179,2596668,Houseman,Jun-04,Aug-20,Aug-20,KY,2020-08-01,2020.0,8.0,2020-08-01,2020.0,8.0,2019-11-01,2019,11
533180,1481867,Mortgage Processor,May-02,Nov-19,Dec-19,WI,2019-11-01,2019.0,11.0,2019-12-01,2019.0,12.0,2019-10-01,2019,10
533185,628283,Financial Advisor,Sep-05,Aug-20,Aug-20,IN,2020-08-01,2020.0,8.0,2020-08-01,2020.0,8.0,2018-01-01,2018,1
533194,2630568,Marketing Support Professional,May-11,Aug-20,Aug-20,FL,2020-08-01,2020.0,8.0,2020-08-01,2020.0,8.0,2019-10-01,2019,10


In [10]:
all_d=X_train_add

all_d['issue_To_last_pymnt']=all_d['issue_d_trans']-all_d['last_pymnt_d_trans']
all_d['last_credit_pull_d_To_last_pymnt']=all_d['last_credit_pull_d_trans']-all_d['last_pymnt_d_trans']
all_d['issue_To_last_credit_pull_d']=all_d['issue_d_trans']-all_d['last_credit_pull_d_trans']

all_d['issue_To_last_pymnt']=all_d['issue_To_last_pymnt'].map(lambda x:x.days)
all_d['last_credit_pull_d_To_last_pymnt']=all_d['last_credit_pull_d_To_last_pymnt'].map(lambda x:x.days)
all_d['issue_To_last_credit_pull_d']=all_d['issue_To_last_credit_pull_d'].map(lambda x:x.days)


C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [11]:
all_d=X_test_add

all_d['issue_To_last_pymnt']=all_d['issue_d_trans']-all_d['last_pymnt_d_trans']
all_d['last_credit_pull_d_To_last_pymnt']=all_d['last_credit_pull_d_trans']-all_d['last_pymnt_d_trans']
all_d['issue_To_last_credit_pull_d']=all_d['issue_d_trans']-all_d['last_credit_pull_d_trans']

all_d['issue_To_last_pymnt']=all_d['issue_To_last_pymnt'].map(lambda x:x.days)
all_d['last_credit_pull_d_To_last_pymnt']=all_d['last_credit_pull_d_To_last_pymnt'].map(lambda x:x.days)
all_d['issue_To_last_credit_pull_d']=all_d['issue_To_last_credit_pull_d'].map(lambda x:x.days)


C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [12]:
add=['Unnamed: 0.1','issue_To_last_pymnt','last_credit_pull_d_To_last_pymnt','issue_To_last_credit_pull_d']
X_test_add=X_test_add[add]
X_train_add=X_train_add[add]

In [13]:
X_test_add_id=X_test_add['Unnamed: 0.1']
X_train_add_id=X_train_add['Unnamed: 0.1']

X_train_add=X_train_add.drop(columns=['Unnamed: 0.1'])
X_test_add=X_test_add.drop(columns=['Unnamed: 0.1'])

In [14]:
# 標準化
 # 标准化数据
ss = StandardScaler()
X_test_add_std= pd.DataFrame(ss.fit_transform(X_test_add),columns=X_test_add.columns,index=X_test_add.index)
 
    
ss = StandardScaler()
X_train_add_std= pd.DataFrame(ss.fit_transform(X_train_add),columns=X_train_add.columns,index=X_train_add.index)
    

In [15]:
X_test_add_std['Unnamed: 0.1']=X_test_add_id
X_train_add_std['Unnamed: 0.1']=X_train_add_id

In [16]:
X_train_add_std

,issue_To_last_pymnt,last_credit_pull_d_To_last_pymnt,issue_To_last_credit_pull_d,Unnamed: 0.1
0,0.771674,3.049468,-1.818767,99620
1,0.679785,-0.574307,0.856861,1676459
3,1.332197,-0.348772,1.093845,2230019
10,0.774737,-0.498284,0.858772,819438
12,0.587896,2.124518,-1.235862,1486093
...,...,...,...,...
2132770,-1.461228,-0.574307,-0.479042,1482659
2132775,-0.251356,-0.574307,0.275867,739979
2132778,-1.467354,0.811850,-1.528270,1650754
2132784,-1.461228,1.665845,-2.168510,2277769


In [17]:
# 
X_train_more=pd.merge(X_train,X_train_add_std)
X_test_more=pd.merge(X_test,X_test_add_std)

In [18]:
# 補值

# 用 mean 補值
var=X_train_more.columns.tolist()

for i in range(0,len(var)): 
    X_train_more[str(var[i])]=X_train_more[str(var[i])].fillna(X_train_more[str(var[i])].mean())

In [19]:
# 用 mean 補值
var=X_test_more.columns.tolist()

for i in range(0,len(var)): 
    X_test_more[str(var[i])]=X_test_more[str(var[i])].fillna(X_test_more[str(var[i])].mean())


## 
X_train_more
X_test_more
y_train
y_test

# model

In [20]:
# mlp
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import itertools
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import Sequential
import keras
from keras.models import Sequential    #匯入線性堆疊模型之模組
from keras.layers import Dense
from keras import metrics#匯入緊密連接模組

C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [21]:
X_train_more.to_csv(path+'\grade\A\X_train_more.csv')
X_test_more.to_csv(path+'\grade\A\X_test_more.csv')

In [22]:
X_train_more=X_train_more.drop(columns=['Unnamed: 0.1'])
X_test_more=X_test_more.drop(columns=['Unnamed: 0.1'])

In [23]:
y_train=pd.read_csv(path+'/grade/A/y_trainA.csv',index_col=0)
y_test=pd.read_csv(path+'/grade/A/y_testA.csv',index_col=0)

# 處理 y
le = LabelEncoder()
le.fit(y_train['loan_status'].unique())  
y_train['ans']=le.transform(y_train['loan_status'])


In [24]:
# 分 train0 test0
X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2, random_state=0)

# M 01

In [24]:
from imblearn.over_sampling import ADASYN 
X4, y4 = ADASYN().fit_resample(X_train0, y_train0.ans)  

(385496,)

In [26]:
y_train0.ans.shape
y_train0.ans.sum()

16554

In [27]:
y4.shape

(737638,)

In [28]:
y4.sum()

368696

In [39]:
X4.shape

(737638, 70)

In [35]:
X_train0=X4
y_train0=y4
y_train0_0 = keras.utils.to_categorical(y_train0, 2)

In [36]:
from keras import backend as K
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.utils import np_utils

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [32]:
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.utils import np_utils

In [40]:
model = Sequential() #建立模型
model.add(Dense(input_dim = 70, output_dim = 48)) #添加输入层、隐藏层的连接
model.add(Activation('tanh')) #以Relu函数为激活函数

model.add(Dense(input_dim = 48, output_dim = 48)) #添加隐藏层、隐藏层的连接
model.add(Activation('relu')) #以Relu函数为激活函数
model.add(Dropout(0.2))

model.add(Dense(input_dim = 48, output_dim = 36)) #添加隐藏层、隐藏层的连接
model.add(Activation('relu')) #以Relu函数为激活函数
model.add(Dropout(0.2))
model.add(Dense(input_dim = 36, output_dim = 36)) #添加隐藏层、隐藏层的连接
model.add(Activation('relu')) #以Relu函数为激活函数

model.add(Dense(input_dim = 36, output_dim = 12)) #添加隐藏层、隐藏层的连接
model.add(Activation('relu')) #以Relu函数为激活函数
model.add(Dense(input_dim = 12, output_dim = 12)) #添加隐藏层、隐藏层的连接
model.add(Activation('relu')) #以Relu函数为激活函数


model.add(Dense(input_dim = 12, output_dim = 2)) #添加隐藏层、输出层的连接
model.add(Activation('sigmoid')) #以sigmoid函数为激活函数

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=70, units=48)`
  
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=48, units=48)`
  """
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=48, units=36)`
  if __name__ == '__main__':
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=36, units=36)`
  if sys.path[0] == '':
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=36, units=12)`
  from ipykernel import kernelapp as app
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to t

In [41]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
               metrics=[f1_m])

train_history=model.fit(X_train0, y_train0_0, validation_split=0.3, epochs=50, batch_size=200, verbose=2)  


Train on 516346 samples, validate on 221292 samples
Epoch 1/50
 - 8s - loss: 0.1108 - f1_m: 0.9606 - val_loss: 0.1097 - val_f1_m: 0.9535
Epoch 2/50
 - 7s - loss: 0.0732 - f1_m: 0.9762 - val_loss: 0.1043 - val_f1_m: 0.9571
Epoch 3/50
 - 7s - loss: 0.0622 - f1_m: 0.9804 - val_loss: 0.0723 - val_f1_m: 0.9714
Epoch 4/50
 - 7s - loss: 0.0558 - f1_m: 0.9827 - val_loss: 0.0805 - val_f1_m: 0.9707
Epoch 5/50
 - 8s - loss: 0.0519 - f1_m: 0.9839 - val_loss: 0.0910 - val_f1_m: 0.9715
Epoch 6/50
 - 8s - loss: 0.0488 - f1_m: 0.9850 - val_loss: 0.0696 - val_f1_m: 0.9734
Epoch 7/50
 - 8s - loss: 0.0463 - f1_m: 0.9859 - val_loss: 0.0589 - val_f1_m: 0.9778
Epoch 8/50
 - 8s - loss: 0.0447 - f1_m: 0.9865 - val_loss: 0.0866 - val_f1_m: 0.9725
Epoch 9/50
 - 7s - loss: 0.0430 - f1_m: 0.9871 - val_loss: 0.0718 - val_f1_m: 0.9778
Epoch 10/50
 - 7s - loss: 0.0411 - f1_m: 0.9876 - val_loss: 0.0613 - val_f1_m: 0.9801
Epoch 11/50
 - 7s - loss: 0.0402 - f1_m: 0.9882 - val_loss: 0.0530 - val_f1_m: 0.9817
Epoch 12/5

In [42]:
prediction=model.predict_classes(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.9024266936299292
0.9919895407475046


In [45]:
X_test.shape

(120505, 68)

In [47]:
## predict Y
y_pred=model.predict_classes(X_test_more)
y_ans = pd.DataFrame(y_pred,columns=['predict'],index=y_test.index)
y_output=pd.merge(y_test,y_ans,left_index=True,right_index=True)

y_output.to_csv(path+'\grade\A\y_output_MLP_A_02.csv')

# M 02

In [34]:
# 分 train0 test0
X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2, random_state=0)

In [35]:
from imblearn.over_sampling import BorderlineSMOTE
oversample = BorderlineSMOTE()
X3, y3 = oversample.fit_resample(X_train0, y_train0.ans)

In [36]:
print(y_train0.ans.sum())
print(y_train0.ans.shape)
print(y3.sum())
print(y3.shape)

16554
(385496,)
368942
(737884,)


In [37]:
X_train0=X3
y_train0=y3
y_train0_0 = keras.utils.to_categorical(y_train0, 2)

In [38]:
X3.to_csv(path+'\grade\A\X3.csv')
y3.to_csv(path+'\grade\A\y3.csv')

In [39]:
from keras import backend as K
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.utils import np_utils

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [40]:
model01 = Sequential() #建立模型
model01.add(Dense(input_dim = 70, output_dim = 48)) #添加输入层、隐藏层的连接
model01.add(Activation('tanh')) #以Relu函数为激活函数

model01.add(Dense(input_dim = 48, output_dim = 48)) #添加隐藏层、隐藏层的连接
model01.add(Activation('relu')) #以Relu函数为激活函数
model01.add(Dropout(0.2))

model01.add(Dense(input_dim = 48, output_dim = 36)) #添加隐藏层、隐藏层的连接
model01.add(Activation('relu')) #以Relu函数为激活函数
model01.add(Dropout(0.2))
model01.add(Dense(input_dim = 36, output_dim = 36)) #添加隐藏层、隐藏层的连接
model01.add(Activation('relu')) #以Relu函数为激活函数

model01.add(Dense(input_dim = 36, output_dim = 12)) #添加隐藏层、隐藏层的连接
model01.add(Activation('relu')) #以Relu函数为激活函数
model01.add(Dense(input_dim = 12, output_dim = 12)) #添加隐藏层、隐藏层的连接
model01.add(Activation('relu')) #以Relu函数为激活函数


model01.add(Dense(input_dim = 12, output_dim = 2)) #添加隐藏层、输出层的连接
model01.add(Activation('sigmoid')) #以sigmoid函数为激活函数

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=70, units=48)`
  
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=48, units=48)`
  """
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=48, units=36)`
  if __name__ == '__main__':
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=36, units=36)`
  if sys.path[0] == '':
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=36, units=12)`
  from ipykernel import kernelapp as app
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to t

In [41]:
model01.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
               metrics=[f1_m])

train_history=model01.fit(X_train0, y_train0_0, validation_split=0.3, epochs=70, batch_size=200, verbose=2)  

Train on 516518 samples, validate on 221366 samples
Epoch 1/70
 - 8s - loss: 0.0809 - f1_m: 0.9743 - val_loss: 0.0490 - val_f1_m: 0.9808
Epoch 2/70
 - 8s - loss: 0.0498 - f1_m: 0.9853 - val_loss: 0.0319 - val_f1_m: 0.9865
Epoch 3/70
 - 8s - loss: 0.0434 - f1_m: 0.9874 - val_loss: 0.0275 - val_f1_m: 0.9894
Epoch 4/70
 - 7s - loss: 0.0406 - f1_m: 0.9885 - val_loss: 0.0285 - val_f1_m: 0.9891
Epoch 5/70
 - 8s - loss: 0.0388 - f1_m: 0.9892 - val_loss: 0.0211 - val_f1_m: 0.9921
Epoch 6/70
 - 8s - loss: 0.0373 - f1_m: 0.9897 - val_loss: 0.0259 - val_f1_m: 0.9900
Epoch 7/70
 - 8s - loss: 0.0360 - f1_m: 0.9900 - val_loss: 0.0237 - val_f1_m: 0.9919
Epoch 8/70
 - 8s - loss: 0.0354 - f1_m: 0.9905 - val_loss: 0.0261 - val_f1_m: 0.9911
Epoch 9/70
 - 8s - loss: 0.0343 - f1_m: 0.9907 - val_loss: 0.0155 - val_f1_m: 0.9939
Epoch 10/70
 - 8s - loss: 0.0337 - f1_m: 0.9909 - val_loss: 0.0212 - val_f1_m: 0.9924
Epoch 11/70
 - 7s - loss: 0.0329 - f1_m: 0.9910 - val_loss: 0.0151 - val_f1_m: 0.9942
Epoch 12/70

In [42]:
prediction=model01.predict_classes(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.8732544221306024
0.9901114408450412


# M 3

In [25]:
# 分 train0 test0
X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2, random_state=0)

In [26]:
from imblearn.over_sampling import SVMSMOTE
oversample = SVMSMOTE()
X1, y1 = oversample.fit_resample(X_train0, y_train0.ans)

In [27]:
X1.to_csv(path+'\grade\A\X1.csv')
y1.to_csv(path+'\grade\A\y1.csv')

In [28]:
print(y_train0.ans.sum())
print(y_train0.ans.shape)
print(y1.sum())
print(y1.shape)

16554
(385496,)
368942
(737884,)


In [29]:
X_train0=X1
y_train0=y1
y_train0_0 = keras.utils.to_categorical(y_train0, 2)

In [30]:
from keras import backend as K
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.utils import np_utils

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [31]:
model02 = Sequential() #建立模型
model02.add(Dense(input_dim = 70, output_dim = 48)) #添加输入层、隐藏层的连接
model02.add(Activation('tanh')) #以Relu函数为激活函数

model02.add(Dense(input_dim = 48, output_dim = 48)) #添加隐藏层、隐藏层的连接
model02.add(Activation('relu')) #以Relu函数为激活函数
model02.add(Dropout(0.2))

model02.add(Dense(input_dim = 48, output_dim = 36)) #添加隐藏层、隐藏层的连接
model02.add(Activation('relu')) #以Relu函数为激活函数
model02.add(Dropout(0.2))
model02.add(Dense(input_dim = 36, output_dim = 36)) #添加隐藏层、隐藏层的连接
model02.add(Activation('relu')) #以Relu函数为激活函数

model02.add(Dense(input_dim = 36, output_dim = 12)) #添加隐藏层、隐藏层的连接
model02.add(Activation('relu')) #以Relu函数为激活函数
model02.add(Dense(input_dim = 12, output_dim = 12)) #添加隐藏层、隐藏层的连接
model02.add(Activation('relu')) #以Relu函数为激活函数


model02.add(Dense(input_dim = 12, output_dim = 2)) #添加隐藏层、输出层的连接
model02.add(Activation('sigmoid')) #以sigmoid函数为激活函数

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=70, units=48)`
  
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=48, units=48)`
  """
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=48, units=36)`
  if __name__ == '__main__':
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=36, units=36)`
  if sys.path[0] == '':
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=36, units=12)`
  from ipykernel import kernelapp as app
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to t

In [32]:
model02.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
               metrics=[f1_m])

train_history=model02.fit(X_train0, y_train0_0, validation_split=0.3, epochs=70, batch_size=200, verbose=2)  

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 516518 samples, validate on 221366 samples
Epoch 1/70
 - 8s - loss: 0.0898 - f1_m: 0.9700 - val_loss: 0.1200 - val_f1_m: 0.9511
Epoch 2/70
 - 7s - loss: 0.0566 - f1_m: 0.9829 - val_loss: 0.1355 - val_f1_m: 0.9446
Epoch 3/70
 - 7s - loss: 0.0480 - f1_m: 0.9859 - val_loss: 0.1046 - val_f1_m: 0.9513
Epoch 4/70
 - 7s - loss: 0.0437 - f1_m: 0.9874 - val_loss: 0.2595 - val_f1_m: 0.8837
Epoch 5/70
 - 7s - loss: 0.0405 - f1_m: 0.9883 - val_loss: 0.2456 - val_f1_m: 0.8889
Epoch 6/70
 - 7s - loss: 0.0391 - f1_m: 0.9888 - val_loss: 0.0740 - val_f1_m: 0.9710
Epoch 7/70
 - 7s - loss: 0.0375 - f1_m: 0.9895 - val_loss: 0.2155 - val_f1_m: 0.9096
Epoch 8/70
 - 7s - loss: 0.0369 - f1_m: 0.9898 - val_loss: 0.1219 - val_f1_m: 0.9535
Epoch 9/70
 - 7s - loss: 0.0355 - f1_m: 0.9904 - val_loss: 0.1474 - val_f1_m: 0.9405
Epoch 10/70
 - 7s - loss: 0.0353 - f1_m: 0.9906 - val_loss: 0.2157 - val_f1_m: 0.9135
Ep

In [33]:
prediction=model02.predict_classes(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.888465204957102
0.990287836968477


In [ ]:
## predict Y
y_pred=model02.predict_classes(X_test_more)
y_ans = pd.DataFrame(y_pred,columns=['predict'],index=y_test.index)
y_output=pd.merge(y_test,y_ans,left_index=True,right_index=True)

y_output.to_csv(path+'\grade\A\y_output_MLP_A_04.csv')